# Spatial Taxi Pickup Intensity (KDE replication)

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.neighbors import KernelDensity

BASE_DIR = Path.cwd()
for candidate in [BASE_DIR, *BASE_DIR.parents]:
    if (candidate / 'src').exists():
        BASE_DIR = candidate
        break
else:
    raise FileNotFoundError('Could not locate src directory')

if str(BASE_DIR / 'src') not in sys.path:
    sys.path.append(str(BASE_DIR / 'src'))

DATA_DIR = BASE_DIR / 'data' / 'raw'
lookup_path = DATA_DIR / 'taxi_zone_lookup.csv'
YELLOW_PATH = DATA_DIR / 'yellow_tripdata_2024-01.parquet'

In [ ]:
from modeling.poisson_zone import load_taxi_pickups

In [ ]:
lookup = pd.read_csv(lookup_path)
poisson_sample = load_taxi_pickups(YELLOW_PATH, max_rows=500_000)
poisson_sample = poisson_sample.merge(lookup[["LocationID","Zone","service_zone"]], left_on="PULocationID", right_on="LocationID", how="left")

## Convert to coordinates and select Manhattan trips

In [ ]:
import re

def extract_point(wkt):
    m = re.search(r'POINT \(([-0-9.]+) ([-0-9.]+)\)', str(wkt))
    if m:
        return float(m.group(1)), float(m.group(2))
    return None, None

lookup[['lon','lat']] = lookup['the_geom'].apply(lambda g: pd.Series(extract_point(g)))
trips = poisson_sample.merge(lookup[['LocationID','lon','lat']], left_on='PULocationID', right_on='LocationID', how='left')
trips = trips.dropna(subset=['lon','lat'])
trips = trips[trips['service_zone']=='Yellow Zone']
trips.head()

## KDE estimation

In [ ]:
sample = trips[['lon','lat','tpep_pickup_datetime']].copy()
sample['hour'] = pd.to_datetime(sample['tpep_pickup_datetime']).dt.hour
hour_subset = sample[sample['hour'].between(7,9)]
coords = hour_subset[['lon','lat']].values
kde = KernelDensity(bandwidth=0.01, kernel='gaussian').fit(coords)
# grid
lon_lin = np.linspace(coords[:,0].min(), coords[:,0].max(), 100)
lat_lin = np.linspace(coords[:,1].min(), coords[:,1].max(), 100)
xx, yy = np.meshgrid(lon_lin, lat_lin)
grid_samples = np.vstack([xx.ravel(), yy.ravel()]).T
log_dens = kde.score_samples(grid_samples)
dens = np.exp(log_dens).reshape(xx.shape)
fig = px.imshow(dens, origin='lower', x=lon_lin, y=lat_lin, color_continuous_scale='Oranges')
fig.update_layout(title='KDE intensity (7-9am)')
fig.show()